### Inventory Scoring Mode

#### Import Packages

In [32]:
import pandas as pd
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from category_encoders import OrdinalEncoder

#### Load the datasets

In [33]:
campaign_df = pd.read_csv("../data/campaign_data.csv")
creative_df = pd.read_csv("../data/creative_data.csv")
inventory_df = pd.read_csv("../data/Inventory_data.csv")

##### Fix inconsistency namings

In [34]:
campaign_df.rename(columns={'campaignid': 'campaign_id'}, inplace=True)

##### Merge the 3 dataframes based on campaign_ig and game_key

In [35]:
# Merge the dataframes based on common columns
merged_df = pd.merge(campaign_df, creative_df, on='campaign_id', how='left')
merged_df = pd.merge(merged_df, inventory_df, on=['campaign_id', 'game_key'], how='left')

merged_df.head()

,campaign_id,budget_amount,budget_currencycode,KPI,pricing_model,geo_targeting,vertical,targeting,game_key,adformat,...,renderingcontext,osfamily,devicetype,devicemake,site_name,matchedfoldposition,browser,impression,engagement,click
0,f4p5a01,277.12,USD,Engagement,CPE,NaN,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,320x480,...,MobileOptimizedWeb,6.0,3.0,Samsung,www.yahoo.com,4.0,Chrome,57.0,0.0,0.0
1,f4p5a01,277.12,USD,Engagement,CPE,NaN,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,320x480,...,MobileOptimizedWeb,5.0,4.0,Apple,www.yahoo.com,4.0,Mobile Safari,983.0,265.0,71.0
2,f4p5a01,277.12,USD,Engagement,CPE,NaN,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,320x480,...,MobileOptimizedWeb,6.0,4.0,Samsung,www.yahoo.com,4.0,Chrome,9.0,0.0,0.0
3,f4p5a01,277.12,USD,Engagement,CPE,NaN,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,320x480,...,MobileOptimizedWeb,5.0,4.0,Apple,www.the-sun.com,4.0,Mobile Safari UI/WKWebView,3.0,0.0,0.0
4,f4p5a01,277.12,USD,Engagement,CPE,NaN,Sports,Opinion Leaders| Investors| Policy Makers| C&I...,b4f31c796c130fd594d253ba9336a01d/7ae9931edb2ed...,320x480,...,MobileOptimizedWeb,6.0,4.0,Samsung,currently.att.yahoo.com,4.0,Chrome Mobile,27.0,3.0,0.0


#### Remove rows that have a campaign but no inventory

In [36]:
merged_df = merged_df[~merged_df['campaign_id'].isin(campaign_df[~campaign_df['campaign_id'].isin(inventory_df['campaign_id'])]['campaign_id'])]

#### Compute click through rate

In [37]:
merged_df['CTR'] = np.where(merged_df['impression'] != 0,
                            (merged_df['click'] / merged_df['impression']) * 100,
                            0)

#### Mapping of number to value for 3 colums

In [38]:
os_family_mapping = { 1: "Other", 2: "Windows",  3: "OS X",  4: "Linux",  5: "iOS", 6: "Android",  7: "Windows Phone"}
device_type_mapping = { 1: "Other",2: "PC", 3: "Tablet", 4: "Mobile", 5: "Roku", 6: "ConnectedTV", 7: "OutOfHome", 8: "HomeAssistant"}
foldposition_mapping = {1: "Any", 2: "Above", 3: "Below", 4: "Unknown"}

merged_df["osfamily"] = merged_df["osfamily"].map(os_family_mapping)
merged_df["devicetype"] = merged_df["devicetype"].map(device_type_mapping)
merged_df["matchedfoldposition"] = merged_df["matchedfoldposition"].map( foldposition_mapping)

In [40]:
merged_df = merged_df.drop(columns=['campaign_id', "budget_amount", "budget_currencycode", "KPI", "pricing_model", "game_key", "impression", "engagement", "click", "site_name", "targeting"])

In [41]:
merged_df.columns

Index(['geo_targeting', 'vertical', 'adformat', 'creative', 'renderingcontext',
       'osfamily', 'devicetype', 'devicemake', 'matchedfoldposition',
       'browser', 'CTR'],
      dtype='object')

In [42]:
merged_df.head(1).to_dict(orient="records")

[{'geo_targeting': nan,
  'vertical': 'Sports',
  'adformat': '320x480',
  'creative': 'Sensory Video',
  'renderingcontext': 'MobileOptimizedWeb',
  'osfamily': 'Android',
  'devicetype': 'Tablet',
  'devicemake': 'Samsung',
  'matchedfoldposition': 'Unknown',
  'browser': 'Chrome',
  'CTR': 0.0}]

#### Handling Missing Data 

In [43]:
# Calculate the percentage of missing values in each column
missing_percentage = merged_df.isnull().sum() / len(merged_df) * 100

# Print the percentage of missing values for each column
print(missing_percentage)

geo_targeting          91.161239
vertical                0.000000
adformat                0.000000
creative                2.900846
renderingcontext        0.000000
osfamily                0.003899
devicetype              0.000101
devicemake              1.916644
matchedfoldposition     0.000067
browser                 0.091890
CTR                     0.000000
dtype: float64


In [45]:
merged_df['geo_targeting'].fillna(value="Unknown", inplace=True)
# merged_df['targeting'].fillna(value="Unknown", inplace=True)
merged_df['creative'].fillna(value="Unknown", inplace=True)

merged_df['browser'].fillna(value=merged_df['browser'].mode()[0], inplace=True)
merged_df['devicemake'].fillna(value=merged_df['devicemake'].mode()[0], inplace=True)
merged_df['devicetype'].fillna(value=merged_df['devicetype'].mode()[0], inplace=True)
merged_df['matchedfoldposition'].fillna(value=merged_df['matchedfoldposition'].mode()[0], inplace=True)
merged_df['osfamily'].fillna(value=merged_df['osfamily'].mode()[0], inplace=True)

In [13]:
sample_df = merged_df.iloc[:, :]

In [59]:
# Optional: Encode categorical variables if they're not numerically encoded
# It's important for algorithms like XGBoost that expect numerical input
encoder = OrdinalEncoder(cols=['geo_targeting', 'vertical', 'targeting', 'adformat', 'creative',
                               'renderingcontext', 'osfamily', 'devicetype', 'devicemake', 
                               'site_name', 'matchedfoldposition', 'browser'])
merged_df_encoded = encoder.fit_transform(sample_df)



In [60]:
# Split data into features and target
X = merged_df_encoded.drop('CTR', axis=1)
y = merged_df_encoded['CTR']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [65]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assuming merged_df is your DataFrame

# Define categorical columns
categorical_columns = ['matchedfoldposition', 'devicetype', 'osfamily']

# Map categorical columns to their categories
categories = {
    'matchedfoldposition': ['Any', 'Above', 'Below', 'Unknown'],
    'devicetype': ['Other', 'PC', 'Tablet', 'Mobile', 'Roku', 'ConnectedTV', 'OutOfHome', 'HomeAssistant'],
    'osfamily': ['Other', 'Windows', 'OS X', 'Linux', 'iOS', 'Android', 'Windows Phone']
}

# Preprocessing for categorical data
categorical_preprocessor = OneHotEncoder(categories=[categories[col] for col in categorical_columns])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_preprocessor, categorical_columns)
    ], remainder='passthrough')

In [70]:
# Define the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[
                      ('model', model)])

In [71]:
# Separate target from predictors
X = merged_df.drop('CTR', axis=1)
y = merged_df['CTR']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

In [32]:
sample_df = merged_df.iloc[:, :]

In [38]:
sample_df.iloc[:1,:1].dtypes

geo_targeting    object
dtype: object

In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Assuming 'merged_df' is your DataFrame and 'CTR' is your target

# Splitting features and target
X = sample_df.drop(['CTR'], axis=1)
y = sample_df['CTR']

# Identifying categorical columns (excluding target variable 'CTR')
categorical_cols = ["matchedfoldposition", "devicetype", "osfamily"]

# No explicit numerical features to preprocess in this dataset except the target
# If there were numerical features, they would be listed here

# Preprocessor for categorical data with OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])


#### Random Forest Regressor

In [18]:
# Defining the model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the model
model.fit(X_train, y_train)

# Predicting CTR
predictions = model.predict(X_test)

# Calculating the Root Mean Square Error (RMSE)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")


RMSE: 15.923724654407854


#### Gradient Boosting Regressor

In [19]:
from sklearn.ensemble import GradientBoostingRegressor

# Updating the model in the pipeline to GradientBoostingRegressor
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(n_estimators=100, random_state=42))
])

# Training the model
model.fit(X_train, y_train)

# Predicting CTR
predictions = model.predict(X_test)

# Calculating the RMSE
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 15.923745907743601


#### XGBoost (eXtreme Gradient Boosting) l

In [20]:
from xgboost import XGBRegressor

# Updating the model in the pipeline to XGBRegressor
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, random_state=42))
])

# Training the model
model.fit(X_train, y_train)

# Predicting CTR
predictions = model.predict(X_test)

# Calculating the RMSE
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 15.923730551651596


#### LightGBM (Light Gradient Boosting Machine)

In [22]:
from lightgbm import LGBMRegressor

# Define a new model pipeline with LightGBM
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LGBMRegressor(objective='regression', num_leaves=31, learning_rate=0.05, n_estimators=100, random_state=42))
])

# Splitting the dataset (ensure X and y are already defined as per your dataset)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the model
model.fit(X_train, y_train)

# Predicting and evaluating
predictions = model.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.082810 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28
[LightGBM] [Info] Number of data points in the train set: 2380243, number of used features: 14
[LightGBM] [Info] Start training from score 3.819118
RMSE: 15.92369691961791
